<a href="https://colab.research.google.com/github/iamchenchu/Deep-Learnig-with-TensorFlow/blob/main/09_Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP(Naturala Language Processing) and NLU (Natural Language Understanding) :

`A handful of example natural language processing (NLP) and natural language understanding (NLU) problems. These are also often referred to as sequence problems (going from one sequence to another)`

NLP has the goal of deriving information out of natural language (Could be sequences text or speech). Another common term of NLP probelems is sequence to sequence problems (seq2seq)

**Let's see where do we use it : Which are reffered to as a Sequence Problems**

1. MultiLabel Classification
2. Machine Translation
3. Text Generation
4. Voice Assistant

**Types of Sequence problems :(input to output types)**
1. One to one  (one word translation)
2. One to many (Image captioning, possible to have multiple captions)
3. Many to one (Sentiment analysis maybe a youtube comment, BitCoin Price Prediction)
4. many to many (Part-of-speech tagging in natural language processing. Each word in the input sentence may correspond to multiple possible part-of-speech tags.)
5. many to many (DNA sequence alignment. Aligning two DNA sequences where each base in one sequence may correspond to multiple bases in the other.)







# What will we do in this section :         

* Downloading and preparing the text data
* How to prepare text data for modeling (Tokenization and embedding)
* Setting up multiple modeling experiments with Recurrent Neural Networks (RNN)
* Building a text feature extraction model using TensorFlow Hub
* Finding the most wrong prediction examples
* Using a model we've built to make predictions on the text from the wild



**NLP Inputs and outputs :**

`Text -> turn into numbers -> build a model -> train the model to find patterns -> use patterns (make predictions)`

esource: For a great overview of NLP and the different problems within it, read the article A Simple Introduction to [Natural Language Processing](https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32).



# STEPS  WE FOLLOW IN CREATING IN RNN / CNN FOR NLP

1. Get the data ready (Turn all data into numbers as neural networks can't handle text or natural language and make sure all of your tensors are in right shape pad sequence which don't fit )
2. Build or pick a pretrained model (to suit your problem)
3. Fit the model to the data and make a prediction
4. Evaluate the model
5. Improve through Experimentation
6. Save and reload your trained model



# TYPICAL ARCHITECTURE OF A RECCURENT NEURAL NETWORK

**What is RNN :**  is class of Artificial Neural Networks where connections between nodes from a directed graph along a temporal sequence. This allows it to exhibit temporal dynamic behaviour.

1. Input Text
2. Input Layers
3. Text vectorization layer
4. Embedding (Turns mapping of text vectors to embedding matrix (representing how words relate Ex King-man+woman = queen))
5. RNN Cells (Finds Pattern in sequence) `Ex : Simple RNN, LSTM, GRU`
6. Hidden Activations : Adds non linearity to learned features usually use `tanh`
7. Pooling layer : Reduces the dimensionality of the learned features usually Conv1D models `Ex : GlobalAveragePooling1D or GlobalMaxPool1D`
8. Fully Connected layer : Further refines learned features from recurrent layers (Dense layer)
9. Output layer (Takes learned feature and outputs them in shape of target labels) Ex : `output_shape = [number_of_classes] (Disaster or not)`
10. Output Activation : Adds non-linearities to output layer.






# Check GPU

In [2]:
!nvidia-smi

Thu Dec 28 03:47:05 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Get helper functions

In [4]:
# Get helper functions Import the series of the helper functions using the below link

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#Import the functions using import
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-28 03:49:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-28 03:49:58 (108 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



# Get the Text Dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as diaster or not diaster)

See the original source here :https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip



In [5]:
#Get the data

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip the data
unzip_data("nlp_getting_started.zip")

--2023-12-28 03:53:34--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.207, 142.251.10.207, 142.251.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   726KB/s    in 0.8s    

2023-12-28 03:53:35 (726 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Becoming one with the data and visualizing a text dataset

